# KDD Cup 1999 - Keşifsel Veri Analizi (EDA)

Bu notebook, KDD Cup 1999 veri setinin keşifsel veri analizini içerir.

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data import load_kdd, KDD_COLS, ATTACK_FAMILY
from src.preprocess import add_targets, split_features
from src.viz import boxplots, plot_attack_distribution, plot_service_attack_ratio, plot_correlation_heatmap

# Görselleştirme ayarları
plt.style.use('default')
sns.set_palette('husl')
%matplotlib inline

## 1. Veri Yükleme

**Not:** Veri dosyalarını `data/` klasörüne yerleştirdiğinizden emin olun.

In [ ]:
# Veri yükleme (dosya yollarını kendi sisteminize göre ayarlayın)
try:
    train_raw = load_kdd('../data/kddcup.data_10_percent.gz')
    test_raw = load_kdd('../data/corrected.gz')
    print(f"Train veri boyutu: {train_raw.shape}")
    print(f"Test veri boyutu: {test_raw.shape}")
except FileNotFoundError:
    print("Veri dosyaları bulunamadı. Lütfen data/ klasörüne KDD Cup 1999 dosyalarını yerleştirin.")

## 2. Temel Veri İncelemesi

In [ ]:
# Temel bilgiler
print("Kolon isimleri:")
print(train_raw.columns.tolist())
print(f"\nToplam kolon sayısı: {len(train_raw.columns)}")

# İlk birkaç satır
print("\nİlk 5 satır:")
train_raw.head()

In [ ]:
# Veri tipleri ve eksik değerler
print("Veri tipleri:")
print(train_raw.dtypes)

print("\nEksik değer sayıları:")
print(train_raw.isnull().sum().sum())

print("\nTemel istatistikler:")
train_raw.describe()

## 3. Hedef Değişken Analizi

In [ ]:
# Hedef değişkenleri ekle
train = add_targets(train_raw)
test = add_targets(test_raw)

# Label dağılımı
print("Train set - Label dağılımı:")
print(train['label'].value_counts())

print("\nTrain set - Binary dağılım:")
print(train['y_binary'].value_counts())

print("\nTrain set - Attack family dağılımı:")
print(train['y_family'].value_counts())

In [ ]:
# Saldırı dağılımını görselleştir
plot_attack_distribution(train)

## 4. Kategorik Değişken Analizi

In [ ]:
# Kategorik değişkenlerin benzersiz değer sayıları
categorical_cols = ['protocol_type', 'service', 'flag']

for col in categorical_cols:
    print(f"{col}: {train[col].nunique()} benzersiz değer")
    print(train[col].value_counts().head(10))
    print("" + "-"*50)

In [ ]:
# Servis bazında saldırı oranları
plot_service_attack_ratio(train, top_n=15)

## 5. Sayısal Değişken Analizi

In [ ]:
# Sıfır varyanslı kolonları tespit et
numeric_cols = train.select_dtypes(include=[np.number]).columns
zero_variance_cols = []

for col in numeric_cols:
    if train[col].nunique() <= 1:
        zero_variance_cols.append(col)

print(f"Sıfır varyanslı kolonlar: {zero_variance_cols}")

In [ ]:
# Önemli sayısal değişkenler için boxplot
important_cols = ['duration', 'src_bytes', 'dst_bytes', 'count', 'srv_count', 'dst_host_count']
boxplots(train, important_cols)

## 6. Korelasyon Analizi

In [ ]:
# Korelasyon ısı haritası
plot_correlation_heatmap(train)

## 7. Aykırı Değer Analizi

In [ ]:
from src.viz import zscore_outliers

# Z-score ile aykırı değerleri tespit et
outlier_cols = ['src_bytes', 'dst_bytes', 'count', 'srv_count']
outliers, z_scores = zscore_outliers(train, outlier_cols, threshold=3.0)

print(f"Toplam aykırı değer sayısı: {len(outliers)}")
print(f"Toplam veri sayısına oranı: {len(outliers)/len(train)*100:.2f}%")

## 8. Tekrar Eden Kayıtlar

In [ ]:
# Tekrar eden kayıtları kontrol et
duplicates = train.duplicated().sum()
print(f"Tekrar eden kayıt sayısı: {duplicates}")
print(f"Toplam veriye oranı: {duplicates/len(train)*100:.2f}%")

## 9. Özet ve Sonuçlar

Bu EDA sonucunda elde edilen temel bulgular:

1. **Veri Boyutu**: Train ve test setlerinin boyutları
2. **Sınıf Dengesizliği**: Normal vs saldırı oranları
3. **Kategorik Değişkenler**: Protocol, service, flag değişkenlerinin dağılımları
4. **Sayısal Değişkenler**: Aykırı değerler ve dağılımlar
5. **Veri Kalitesi**: Eksik değerler, tekrar eden kayıtlar, sıfır varyanslı kolonlar

**Sonraki Adımlar:**
- Binary sınıflandırma modeli geliştirme
- Multi-class sınıflandırma modeli geliştirme
- Model performanslarının karşılaştırılması